In [23]:
import sagemaker, boto3

session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=session)
sagemaker_client = session.client('sagemaker')
role = sagemaker.get_execution_role()

In [24]:
import time, os, sys
import numpy as np
import pandas as pd
import itertools

from pprint import pprint

In [25]:
bucket_name = sagemaker_session.default_bucket()

In [26]:
!pip install sagemaker-experiments

In [27]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [28]:
import random
import string

def generate_random_string():
    list_of_chars = random.choices(string.ascii_uppercase, k=10)
    return ''.join(list_of_chars)

In [29]:
training_experiment = Experiment.create(
    experiment_name = f"experiment-{generate_random_string()}",
    description     = "Experiment Description",
    sagemaker_boto_client=sagemaker_client)

In [30]:
hyperparam_options = {
    'max_depth': [2, 8],
    'eta': [0.2],
    'gamma': [3, 4],
    'min_child_weight': [6],
    'subsample': [0.4],
    'num_round': [10, 20],
    'objective': ['binary:logistic']
}

In [31]:
def prepare_hyperparam_variations(options):
    names, values = zip(*options.items())
    return [dict(zip(names, value)) for value in itertools.product(*values)]

In [32]:
hyperparam_variations = prepare_hyperparam_variations(hyperparam_options)
hyperparam_variations

[{'max_depth': 2,
  'eta': 0.2,
  'gamma': 3,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 10,
  'objective': 'binary:logistic'},
 {'max_depth': 2,
  'eta': 0.2,
  'gamma': 3,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 20,
  'objective': 'binary:logistic'},
 {'max_depth': 2,
  'eta': 0.2,
  'gamma': 4,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 10,
  'objective': 'binary:logistic'},
 {'max_depth': 2,
  'eta': 0.2,
  'gamma': 4,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 20,
  'objective': 'binary:logistic'},
 {'max_depth': 8,
  'eta': 0.2,
  'gamma': 3,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 10,
  'objective': 'binary:logistic'},
 {'max_depth': 8,
  'eta': 0.2,
  'gamma': 3,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 20,
  'objective': 'binary:logistic'},
 {'max_depth': 8,
  'eta': 0.2,
  'gamma': 4,
  'min_child_weight': 6,
  'subsample': 0.4,
  'num_round': 10,
  'objective': 'bina

In [33]:
s3_bucket = 'sagemaker-cookbook-bucket'
prefix = "chapter05"

training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training_data.csv" 
validation_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/validation_data.csv" 
test_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/test_data.csv" 
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

In [34]:
from sagemaker.image_uris import retrieve
container = retrieve('xgboost', boto3.Session().region_name, version="0.90-2")
container

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [35]:
from sagemaker.inputs import TrainingInput
    
s3_input_training = TrainingInput(training_s3_input_location, content_type="text/csv")
s3_input_validation = TrainingInput(validation_s3_input_location, content_type="text/csv")

In [36]:
with Tracker.create(display_name="xgboost-experiment-display-name", 
                    artifact_bucket="sagemaker-cookbook-bucket",
                    artifact_prefix=training_experiment.experiment_name,
                    sagemaker_boto_client=sagemaker_client) as experiment_tracker:
    
    experiment_tracker.log_input(name="training-input", 
                                 media_type="s3/uri", 
                                 value=training_s3_input_location)
    
    experiment_tracker.log_input(name="validation-input", 
                                 media_type="s3/uri", 
                                 value=validation_s3_input_location)
    
    experiment_tracker.log_parameters(hyperparam_options)

In [37]:
experiment_tracker.__dict__

{'trial_component': TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fbce18e89b0>,trial_component_name='TrialComponent-2021-04-23-163733-dfmu',display_name='xgboost-experiment-display-name',tags=None,trial_component_arn='arn:aws:sagemaker:us-east-1:581320662326:experiment-trial-component/trialcomponent-2021-04-23-163733-dfmu',response_metadata={'RequestId': 'dc51d788-99d8-42e3-952f-8207ba9badfa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dc51d788-99d8-42e3-952f-8207ba9badfa', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Fri, 23 Apr 2021 16:37:33 GMT'}, 'RetryAttempts': 0},parameters={'max_depth': [2, 8], 'eta': [0.2], 'gamma': [3, 4], 'min_child_weight': [6], 'subsample': [0.4], 'num_round': [10, 20], 'objective': ['binary:logistic']},input_artifacts={'training-input': TrialComponentArtifact(value='s3://sagemaker-cookbook-bucket/chapter05/input/training_data.csv',media_type='s3/uri'), 'validation-input': T

In [38]:
def track_and_generate_config(experiment_tracker, experiment_name, job_name, random_string, hyperparameters):
    tracker_display_name = f"trial-metadata-{random_string}"
    print(f"{label} Create Tracker: {tracker_display_name}")
    
    with Tracker.create(
        display_name=tracker_display_name,
        artifact_bucket="sagemaker-cookbook-bucket",
        artifact_prefix=f"{experiment_name}/{job_name}",
        sagemaker_boto_client=sagemaker_client) as trial_tracker:
        
        trial_tracker.log_parameters(hyperparameters)

    trial_name = f'trial-{random_string}'
    print(f"Create Trial: {trial_name}")
    
    trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=experiment_name,
        sagemaker_boto_client=sagemaker_client)
    
    trial.add_trial_component(experiment_tracker.trial_component)
    time.sleep(1) 
    trial.add_trial_component(trial_tracker.trial_component)
    
    print(f"Prepare Experiment Configuration")

    return {
        "ExperimentName": experiment_name, 
        "TrialName": trial.trial_name,
        "TrialComponentDisplayName": job_name
    }

In [39]:
import time

for index, hyperparameters in enumerate(hyperparam_variations):
    iteration = index + 1
    print(f"Iteration # {iteration}")
    label = f"[Iteration # {iteration}]"
    random_string = generate_random_string()
    job_name = f"job-{random_string}"
    
    print(f"{label} Track and Generate Config")
    experiment_config = track_and_generate_config(
        experiment_tracker=experiment_tracker,
        experiment_name=training_experiment.experiment_name,
        job_name=job_name,
        random_string=random_string,
        hyperparameters=hyperparameters)
    
    time.sleep(1)
    print(f"{label} Initialize Estimator")
    estimator = sagemaker.estimator.Estimator( 
                container,
                role,
                instance_count=1, 
                instance_type='ml.m5.large', 
                output_path=training_s3_output_location, 
                hyperparameters=hyperparameters,
                enable_sagemaker_metrics = True,
                sagemaker_session=sagemaker_session)
    
    print(f"{label} Call fit() function")
    estimator.fit({'train': s3_input_training, 
                   'validation': s3_input_validation},
                   job_name = job_name, 
                   wait=False,
                   experiment_config=experiment_config)

Iteration # 1
[Iteration # 1] Track and Generate Config
[Iteration # 1] Create Tracker: trial-metadata-PUGURJGJBQ
Create Trial: trial-PUGURJGJBQ
Prepare Experiment Configuration


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: job-PUGURJGJBQ


[Iteration # 1] Initialize Estimator
[Iteration # 1] Call fit() function
Iteration # 2
[Iteration # 2] Track and Generate Config
[Iteration # 2] Create Tracker: trial-metadata-ODRMOZOANU
Create Trial: trial-ODRMOZOANU
Prepare Experiment Configuration


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: job-ODRMOZOANU


[Iteration # 2] Initialize Estimator
[Iteration # 2] Call fit() function
Iteration # 3
[Iteration # 3] Track and Generate Config
[Iteration # 3] Create Tracker: trial-metadata-ZLCAISLRUZ
Create Trial: trial-ZLCAISLRUZ
Prepare Experiment Configuration


...

[Iteration # 6] Initialize Estimator
[Iteration # 6] Call fit() function
Iteration # 7
[Iteration # 7] Track and Generate Config
[Iteration # 7] Create Tracker: trial-metadata-RAJKYEBAQV
Create Trial: trial-RAJKYEBAQV
Prepare Experiment Configuration


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: job-RAJKYEBAQV


[Iteration # 7] Initialize Estimator
[Iteration # 7] Call fit() function
Iteration # 8
[Iteration # 8] Track and Generate Config
[Iteration # 8] Create Tracker: trial-metadata-CZXETKZEPP
Create Trial: trial-CZXETKZEPP
Prepare Experiment Configuration


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: job-CZXETKZEPP


[Iteration # 8] Initialize Estimator
[Iteration # 8] Call fit() function


In [40]:
experiment_name = training_experiment.experiment_name
experiment_name

'experiment-GWOBSJYQLI'

In [41]:
%store experiment_name

Stored 'experiment_name' (str)
